### Impact of Q=cst with one solar spot

In [1]:
"""
Created on Tuesday 18/02/2025

by Sophie Stucki (using Jordi Blanco's code as reference)

"""

"\nCreated on Tuesday 18/02/2025\n\nby Sophie Stucki (using Jordi Blanco's code as reference)\n\n"

In [2]:
import starsim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.interpolate import CubicSpline
import matplotlib.colors as mcolors



In [3]:
pathdata="/Volumes/SD_phd/starsim_/starsim/results/"
path_grid="/Users/sophiestucki/starsim/starsim/gif/"


In [4]:
ss=starsim.StarSim(conf_file_path='/Volumes/SD_phd/starsim_/starsim/starsim.conf')

print(ss.spot_T_contrast)

663.0


In [ ]:
#Starsim simulation parameters


Q=9


t1=np.linspace(0,rot,num=60)

ss.spot_T_contrast=T_con
ss.facula_T_contrast=T_fac
ss.convective_shift=cs
ss.temperature_photosphere=T_eff
ss.rotation_period=rot
ss.radius=radius
ss.facular_area_ratio=Q
ss.inclination=0


overlap=True
tmin,tmax=t1.min(),t1.max()
k=0
while overlap:
    Nspots=1#np.random.randint(15,20)
    longitude=[90,270]
    ss.spot_map=np.zeros([Nspots,7])
    for j in range(Nspots):
        ss.spot_map[j][1]=100#lifetime spot
        ss.spot_map[j][0]=0#appearance time
        ss.spot_map[j][2]=90#latitude (degrees) [0,180]
        ss.spot_map[j][3]=longitude[j]#longitude (degrees)	[0,360]
        ss.spot_map[j][4]=5#spot size (degrees)
    overlap=starsim.nbspectra.check_spot_overlap(ss.spot_map,Q) #true if spots are overlapping
    #checks if the are overlapping spots. If true, find another set of spot parameters

In [6]:
ss.compute_forward(observables=['rv','lc'],t=t1)

Date 28.0. ff_ph=99.561%. ff_sp=0.014%. ff_fc=0.425%. ff_pl=0.000%. [60/60]%00%. [59/60]%%% 

In [7]:
rv_sim=ss.results['rv']
CCF=ss.results['CCF'][1:]
RV=ss.results['CCF'][0]
lc = ss.results['lc']
BIS = ss.results['bis']
'''
Generate an animated GIF
'''

function1 =CCF[:]-np.mean(CCF,axis=0)

x1=RV[:]
lags = np.arange(len(t1))

In [ ]:
# Define the colors
colors  = ["dodgerblue","r"]  # Red -> Black -> Blue

# Create a colormap from the list of colors
colormap = mcolors.LinearSegmentedColormap.from_list("BlueRed", colors)

# Normalize the continuous variable to map it to the range [0, 1] for the colormap
norm = Normalize(vmin=np.min(rv_sim), vmax=np.max(rv_sim))
scalar_map = ScalarMappable(norm=norm, cmap=colormap)

fig, ax = plt.subplots(ncols=5,figsize=(50/1.5, 10/1.5))

def update(frame):
    ax[0].cla()
    ax[1].cla()
    lag = lags[frame]
    
    #plot RVs
    for i in range(len(t1)):
        color_val = scalar_map.to_rgba(rv_sim[i])
        ax[1].plot(t1[i],rv_sim[i], color=color_val,marker='.')
    ax[1].set_xlim(t1.min(),t1.max())
    ax[1].set_ylim(rv_sim.min()-np.abs(rv_sim.min())/10,rv_sim.max()+np.abs(rv_sim.max())/10)
    ax[1].plot(t1[:lag],rv_sim[:lag],color='gray')
    
    #plot CCF
    color_val = scalar_map.to_rgba(rv_sim[lag])
    ax[0].plot(x1,function1[lag], color=color_val,alpha=1)
    for i in range(lag):
        color_val = scalar_map.to_rgba(rv_sim[i])
        ax[0].plot(x1,function1[i], color=color_val,alpha=0.1)

    #plot BIS
    color_val = scalar_map.to_rgba(rv_sim[lag])
    ax[4].plot(rv_sim, BIS, marker='.', color=color_val,alpha=1)


    #plot flux intensity
    for i in range(len(t1)):
        color_val = scalar_map.to_rgba(rv_sim[i])
        ax[3].plot(t1[i],lc[i], color=color_val,marker='.')
    ax[3].set_xlim(t1.min(),t1.max())
    ax[3].plot(t1[:lag],lc[:lag],color='gray')
        
    #generate stellar grid
    x=np.linspace(-0.999,0.999,1000)
    h=np.sqrt((1-x**2)/(np.tan(0)**2+1))
    color_dict = { 0:'red', 1:'black', 2:'yellow', 3:'blue'}
    #0: photosphere, 1: spot, 2: faculae, 3: planet
    
    vec_grid=np.load(path_grid+'vec_gridt{:.4f}.npy'.format(t1[lag]))
    typ=np.load(path_grid+'typt{:.4f}.npy'.format(t1[lag]))
    
    #identifies which type of grid element is covering the pixel: photosphere, spot, facualae or planet
    ax[2].scatter(vec_grid[:,1],vec_grid[:,2], color=[ color_dict[np.argmax(i)] for i in typ ],s=50)
    ax[2].plot(x,h,'k')
    
    ax[0].set_ylabel('CCF power')
    ax[0].set_xlabel('RV (m/s)')
    ax[0].set_ylim(function1.min()-np.abs(function1.min())/10,function1.max()+np.abs(function1.max())/10)
    ax[1].set_ylabel('RV (m/s)')
    ax[1].set_xlabel('t (d)')
    ax[3].set_ylabel('f)')
    ax[3].set_xlabel('t (d)')
    fig.tight_layout()
    
# Create an animation
ani = FuncAnimation(fig, update, frames=len(lags), repeat=False)

# Save the animation as a video
writer = FFMpegWriter(fps=3, metadata=dict(artist='Me'), bitrate=2400)
ani.save(pathdata+"Sun_demo.gif", writer=writer)

MovieWriter stderr:
Error writing trailer of /Volumes/SD_phd/starsim_/starsim/results/Sun_demo.gif: Invalid argument



CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '2400x480', '-pix_fmt', 'rgba', '-framerate', '3', '-loglevel', 'error', '-i', 'pipe:', '-filter_complex', 'split [a][b];[a] palettegen [p];[b][p] paletteuse', '-b', '2400k', '-metadata', 'artist=Me', '-y', '/Volumes/SD_phd/starsim_/starsim/results/Sun_demo.gif']' returned non-zero exit status 255.

In [4]:
from starsim_utils import *

simulation_ini(Q, conf_file_path='/Volumes/SD_phd/starsim_/starsim/starsim.conf', periods_nbr=1)

ModuleNotFoundError: No module named 'starsim_utils'